In [1]:
import sys
import os
from accelerate import notebook_launcher
from datasets import load_dataset
import logging

# Configure logging:
# (i) lear any pre-existing logging handlers (especially useful in notebook reruns)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
# (ii) now safely configure logging
logging.getLogger("codecarbon").setLevel(logging.ERROR)
logging.basicConfig(level=logging.INFO)

# Adjust paths -> import classes
project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
from classes.experiment_config import ExperimentConfig
from classes.experiment_runner import ExperimentRunner

#import torch.multiprocessing as mp
#mp.set_start_method('spawn', force=True)

In [ ]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder=False,
    task_type="text_generation",
    inference_type="pure_generative",  
    max_input_tokens=512,
    max_output_tokens=512,
    num_input_prompts=100,
    save_outputs=True,
    decode_token_to_text=True,
    gpu_list=[0,1,2,3],
    num_processes=4,
    batching_options={
        "fixed_max_batch_size": 8, # this is max batch size if adaptive batching on; fixed batch size if it's off
        "adaptive_batching": False,
        "adaptive_max_tokens": 512  
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD"
    },
    query_rate=1,
    decoder_temperature=1.0, # NB: needs to be a float
    fp_precision="float16",
    backend="pytorch"
)

# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

# function to run the whole experiment workflow.
def run_experiment_workflow():
    # run experiment
    runner = ExperimentRunner(experiment_config, prompts)
    runner.run_torch()
    
    # If not main process, exit cleanly after run
    if not runner.accelerator.is_main_process:
        return  # do NOT proceed further

    # Only main process continues
    if runner.accelerator.is_main_process:
         # aggregate results
        runner.aggregate_results()
        # save results
        runner.save_experiment_results()  
        
    # could add in script here to delete JSON files if storage an issue
    
    runner.teardown()

    return runner

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment_workflow,
                    num_processes=experiment_config.num_processes) 
                    #terminate_on_error=True) # USE THIS IN accelerate.launch.launch()                 

Launching training on 4 GPUs.
Accelerator set up
Unique experiment id: 0186
TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using pytorch, with precision float16
Original generate method saved.
Model and tokenizer prepared
Entering wait barrier: after model preparation
Exiting wait barrier: after model preparation

INFO:classes.experiment_runner:[Process 2626993] Model is on device: cuda:3


INFO:classes.experiment_runner:[Process 2626992] Model is on device: cuda:2
INFO:classes.experiment_runner:[Process 2626991] Model is on device: cuda:1
INFO:classes.experiment_runner:[Process 2626990] Model is on device: cuda:0


Entering wait barrier: after logging device info
Exiting wait barrier: after logging device info
Original generate method reassigned


INFO:classes.experiment_runner:[Process 2626992] Dummy forward pass complete
INFO:classes.experiment_runner:[Process 2626991] Dummy forward pass complete
INFO:classes.experiment_runner:[Process 2626993] Dummy forward pass complete
INFO:classes.experiment_runner:[Process 2626990] Dummy forward pass complete


...

INFO:classes.experiment_runner:[Process 2626992][GPU 2] saved its energy metrics.
INFO:classes.experiment_runner:Process 3: Energy metrics combined successfully.
INFO:classes.experiment_runner:[Process 2626991][GPU 1] saved its energy metrics.


INFO:classes.experiment_runner:[Process 2626993][GPU 3] saved its energy metrics.
INFO:classes.experiment_runner:Process 0: Energy metrics combined successfully.
INFO:classes.experiment_runner:[Process 2626990][GPU 0] saved its energy metrics.


All local process energy metrics saved
Experiment finished
Destroyed process group successfully


INFO:classes.experiment_runner:Aggregating per-process energy metrics from disk.
INFO:classes.experiment_runner:Aggregated global energy results successfully from disk.
INFO:classes.experiment_runner:Saving experiment results
INFO:classes.experiment_runner:Experiment results saved to results/text_generation_results.json


Starting teardown process...
Process group not initialized.
Emptying CUDA cache...
Running garbage collection...
Teardown process complete.
